<a href="https://colab.research.google.com/github/HayaLababidi/indoor-scenes-Recognition/blob/master/Deep_Learning_Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Dataset or only the Saved features in drive

In [0]:
#Download all files in gdrive folder
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1xAboz-oYGj-KI2DBTdrCA98oi1aEH5lW' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = f['title']
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)

**Download  Dataset from Kaggle, unrar and read dataset**

In [0]:
!ls -lha kaggle.json
!pip install -q kaggle
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c fcis-cs-deeplearningcompetition

In [0]:
!unrar x test.rar
!unrar x train.rar

**Read train data and labels**

In [0]:
#@title Read train data and labels
import cv2
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

n_rows = 299
n_cols = 299
n_training_img = 3138


def read_train_images(dir):
    images=np.zeros(shape=(n_training_img,n_rows,n_cols,3), dtype= np.float16)
    labels=np.zeros(shape=(n_training_img), dtype=np.int8)
    String_labels=[]
    
    i=0
    class_label=0
    for root, dirs, files in os.walk(dir):
        temp_label=''
        for name in files:
            temp_label=root[6:]
            image_file_name=os.path.join(root, name)
            img = cv2.imread(image_file_name, cv2.IMREAD_COLOR)
            if img is not None:
                img = img[:, :, [2, 1, 0]]  # Make it RGB
                img = cv2.resize(img, (n_rows, n_cols))
                img = 2 * (img / 255.0) - 1.0
                images[i]=img
                del img
                labels[i]=class_label
                i=i+1
                
        if temp_label != '':
          print(temp_label)
          print(class_label)
          class_label=class_label+1
          String_labels.append(temp_label)
    return images, String_labels, labels
images, String_labels, labels = read_train_images('train')

#split the data into train and validation sets with default shuffle
X_train, X_Val, y_train, y_Val = train_test_split(images, labels, test_size=0.2, random_state=42)

#y_train_one_hot = tf.one_hot(y_train,10)
#y_Val_one_hot = tf.one_hot(y_Val,10)
#del y_train
#del y_Val
 
#convert all lists to np arrray 
X_train = np.asarray(X_train,dtype=np.float16)
X_Val = np.asarray(X_Val,dtype=np.float16)
y_train = np.asarray(y_train,dtype=np.float16)
y_Val = np.asarray(y_Val,dtype=np.float16)
String_labels = np.asarray(String_labels)

print(X_train.shape, X_Val.shape, y_train.shape, y_Val.shape)

#save the np arrays 
np.save('X_train.npy', X_train)
np.save('X_Val.npy', X_Val)
np.save('y_train.npy', y_train)
np.save('y_Val.npy', y_Val)
np.save('String_labels.npy',String_labels)

#delete all arrays from memory to avoid crash due lack of memory
del images
del labels
del X_train
del X_Val
del y_train
del y_Val
del String_labels

**Read test Data**

In [0]:
#@title Read test Data
import cv2
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot  as plt
n_rows = 299
n_cols = 299
n_testing_img = 772


def read_test_images(dir):
    images=np.zeros(shape=(n_testing_img,n_rows,n_cols,3), dtype= np.float16)
    img_names=[]
    i=0
    
    for img_name in (os.listdir(dir)):
        path = os.path.join(dir, img_name)
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        if img is None:
            img = plt.imread(path, cv2.IMREAD_COLOR)
        if img is not None:
            img = img[:, :, [2, 1, 0]]  # Make it RGB
            img = cv2.resize(img, (n_rows, n_cols))
            img = 2 * (img / 255.0) - 1.0
            images[i]=img
            img_names.append(img_name)
            i=i+1
            del img

    return images,img_names
X_test,img_names = read_test_images('test')

print(X_test.shape)
img_names = np.asarray(img_names)
#save the np arrays 
np.save('X_test.npy', X_test)
np.save('img_names.npy', img_names)

#delete all arrays from memory to avoid crash due lack of memory
del X_test
del img_names


# Data Augmentation

In [0]:
import random
import numpy as np
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util
from skimage.color import rgb2gray
from skimage import exposure
from scipy import ndimage
import os
from matplotlib import pyplot as plt
import numpy as np
from scipy.ndimage import zoom


def clipped_zoom(img):
    zoom_factor=1.25
    h, w = img.shape[:2]
    zoom_tuple = (zoom_factor,zoom_factor,1)#for h and width
    # Bounding box of the zoomed-in region within the input array
    zh = int(np.round(h / zoom_factor))
    zw = int(np.round(w / zoom_factor))
    top = (h - zh) // 2
    left = (w - zw) // 2
    return zoom(img[top:top+zh, left:left+zw], zoom_tuple)
     

def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]

def vertical_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[::-1, :]

def rescale(image_array: ndarray):############################# make it zoom
    return sk.transform.rescale(image_array,1.0 / 2.0)

def color2gray(image_array: ndarray):
    return rgb2gray(image_array)

def rotate90(image_array: ndarray):
    return np.rot90(image_array)

def Blurimage(image_array: ndarray):
    return ndimage.uniform_filter(image_array, size=(5, 3, 1))

image_to_transform = sk.io.imread('train/greenhouse/04_06Greenhouse.jpg')
img=random_noise(image_to_transform)
plt.imshow(img)
plt.show()

img=clipped_zoom(img)
plt.imshow(img)
plt.show()

In [0]:
import random
import numpy as np
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util
from skimage.color import rgb2gray
from skimage import exposure
from scipy import ndimage
import os
from scipy.ndimage import zoom



# our folder path containing some images
folder_path = 'train'
# our folder path containing augmented images
folder_augmentation = 'augmentedImages'

def clipped_zoom(image_array: ndarray):
    zoom_factor=random.uniform(1.5, 4)
    h, w = image_array.shape[:2]
    zoom_tuple = (zoom_factor,zoom_factor,1)#for h and width
    # Bounding box of the zoomed-in region within the input array
    zh = int(np.round(h / zoom_factor))
    zw = int(np.round(w / zoom_factor))
    top = (h - zh) // 2
    left = (w - zw) // 2
    return zoom(image_array[top:top+zh, left:left+zw], zoom_tuple)

def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return clipped_zoom(sk.transform.rotate(image_array, random_degree))

def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]

def vertical_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[::-1, :]

def rescale(image_array: ndarray):
    return sk.transform.rescale(image_array,1.0 / 2.0)

def color2gray(image_array: ndarray):
    return rgb2gray(image_array)

def rotate90(image_array: ndarray):
    return np.rot90(image_array)

def Rescaleintensity(image_array: ndarray):
    v_min, v_max = np.percentile(image_array, (0.2, 99.8))
    return exposure.rescale_intensity(image_array, in_range=(v_min, v_max))

def Blurimage(image_array: ndarray):
    return ndimage.uniform_filter(image_array, size=(3, 3, 1))



def DataAugmentation(Total_num_files_desired):
    all_directories = [name for name in os.listdir(folder_path)\
               if os.path.isdir(os.path.join(folder_path, name))]
    #all_directories=['bakery','bedroom','gym','kitchen']
    for classFolder in all_directories:
        # loop on all files of the folder and build a list of files paths
        images = [os.path.join(os.path.join(folder_path,classFolder), f) for f in os.listdir(os.path.join(folder_path,classFolder)) if os.path.isfile(os.path.join(os.path.join(folder_path,classFolder), f))]
        
        # the number of images to generate
        num_files_desired = Total_num_files_desired-len(images)
        print(classFolder,num_files_desired)
        transformed_total_images=[]
        for img in images:
            try: 
                # random image from the folder
                image_path = img
                # read image as an two dimensional array of pixels
                image_to_transform = sk.io.imread(image_path)

                # dictionary of the transformations functions we defined earlier
                available_transformations = {
                    'random_rotation': random_rotation,
                    #'random_noise': random_noise,
                    'horizontal_flip': horizontal_flip,
                    'clipped_zoom': clipped_zoom,
                    #'vertical_flip': vertical_flip,
                    #'rescale': rescale,
                    #'color2gray': color2gray,
                    #'rotate90': rotate90,
                    #'Rescale intensity': Rescaleintensity,
                    #'Blur image': Blurimage
                }

                # random num of transformations to apply
                #num_transformations_to_apply = random.randint(1, len(available_transformations))

                num_transformations = 0
                transformed_image = None
                # can make 4 transformation in single image
                #while num_transformations < 3:
                for key in available_transformations:
                    # choose a random transformation to apply for a single image
                    #key = random.choice(list(available_transformations))
                    transformed_image = available_transformations[key](image_to_transform)
                    #image_to_transform = transformed_image
                    transformed_total_images.append(image_to_transform)
                    num_transformations += 1
                    if num_files_desired < 100 and num_transformations == 1:
                        #print(key)
                        break
                # define a name for our new file
            except:
               pass
        
        num_generated_files = 0
        random.shuffle(transformed_total_images)
        while num_generated_files < num_files_desired and num_generated_files < len(transformed_total_images):
            try:    
                new_file_path = '%s/augmented_image_%s.jpg' % (os.path.join(folder_augmentation,classFolder), num_generated_files)

                if not os.path.exists(os.path.join(folder_augmentation,classFolder)):
                    os.makedirs(os.path.join(folder_augmentation,classFolder))

                # write image to the disk
                transformed_image = transformed_total_images[num_generated_files]
                sk.io.imsave(new_file_path, transformed_image)
            except:
              pass
            num_generated_files = num_generated_files + 1
        print(num_generated_files)

Total_num_files_desired = 500

DataAugmentation(Total_num_files_desired)

In [0]:
import shutil

!zip -r augmentedImages5.zip augmentedImages

#!unzip augmentedImages4.zip

#shutil.rmtree('augmentedImages/gym')

In [0]:
#@title Read Augmented data and labels
import cv2
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

n_rows = 299
n_cols = 299
n_aug_img = 1756

train_lebels_dictionary={
'airport_inside':	2,
'bakery':	6,
'bedroom':	5,
'greenhouse':	8,
'gym':	0,
'kitchen':	7,
'operating_room':	9,
'poolinside':	3,
'restaurant':	1,
'toystore':	4}

def read_train_images(dir):
    images=np.zeros(shape=(n_aug_img,n_rows,n_cols,3), dtype= np.float16)
    labels=[]
    String_labels=[]
    
    i=0
    #class_label=0
    for root, dirs, files in os.walk(dir):
        temp_label=''
        for name in files:
            temp_label=root[16:]
            image_file_name=os.path.join(root, name)
            img = cv2.imread(image_file_name, cv2.IMREAD_COLOR)
            if img is not None:
                img = img[:, :, [2, 1, 0]]  # Make it RGB
                img = cv2.resize(img, (n_rows, n_cols))
                img = 2 * (img / 255.0) - 1.0
                images[i]=img
                del img
                labels.append(train_lebels_dictionary[temp_label])
                i=i+1
                
        if temp_label != '':
          print(temp_label)
          #print(class_label)
          #class_label=class_label+1
          String_labels.append(temp_label)
    print(i)
    return images, String_labels, labels
images, String_labels, labels = read_train_images('augmentedImages')


#convert all lists to np arrray 
X_aug = np.asarray(images,dtype=np.float16)
y_aug = np.asarray(labels)
String_labels_aug = np.asarray(String_labels)



#save the np arrays 
np.save('X_aug.npy', X_aug)
np.save('y_aug.npy', y_aug)
np.save('String_labels_aug.npy',String_labels_aug)

#delete all arrays from memory to avoid crash due lack of memory
print(X_aug.shape)
print(y_aug.shape)
print(images.shape)
print(len(labels))
del images
del labels
del y_aug
del X_aug
del String_labels

airport_inside
poolinside
operating_room
toystore
greenhouse
restaurant
gym
bakery
1756
(1756, 299, 299, 3)
(1756,)
(1756, 299, 299, 3)
1756


# Model & Feature Extraction

**Dowlnoad Pretrained Model checkpoint**

In [0]:
#@title Download ResNet Model checkpoint
import requests

url = 'http://download.tensorflow.org/models/inception_resnet_v2_2016_08_30.tar.gz'
r = requests.get(url, allow_redirects=True)
open('inception_resnet_v2_2016_08_30.tar.gz', 'wb').write(r.content)
!tar -xvf inception_resnet_v2_2016_08_30.tar.gz

inception_resnet_v2_2016_08_30.ckpt


In [0]:
#to be able to call the function that define the shape of the model 
exec(compile(open("inception_resnet_v2.py", "rb").read(), "inception_resnet_v2.py", 'exec'))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
#@title Restore ResNet Model
# ------------------------------------------------------------------------------
# Imports
# ------------------------------------------------------------------------------
import tensorflow as tf
import inception_resnet_v2 as incep_v2
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # Or from tqdm import tqdm if not jupyter notebook
import matplotlib.patches as mpatches

n_rows = 299
n_cols = 299


# ------------------------------------------------------------------------------
# Declarations
# ------------------------------------------------------------------------------
def define_model(model, is_training):
    model.Image = tf.placeholder(tf.float32, shape=[None, n_rows, n_cols, 3])
    with incep_v2.slim.arg_scope(incep_v2.inception_resnet_v2_arg_scope()):
        model.logits, model.end_points = incep_v2.inception_resnet_v2_base(model.Image)

sess = tf.Session()

class Model_Class:
    def __init__(self, is_training):
        define_model(self, is_training=is_training)


# ------------------------------------------------------------------------------
# Create Model
# ------------------------------------------------------------------------------
model = Model_Class(False)

# ------------------------------------------------------------------------------
# Load the weights and Run the classifier
# ------------------------------------------------------------------------------
model.saver = tf.train.Saver()
model.saver.restore(sess, "inception_resnet_v2_2016_08_30.ckpt")


**Feature Extraction**

In [0]:
#@title Feature Extraction Train Data
import numpy as np
import tflearn

start = 100
end = 2500
step = 100
#(2510, 299, 299, 3)
X_train = np.load('X_train.npy')
#can't do it at once Memory Crash
X_train_features = sess.run(model.logits, feed_dict={model.Image:X_train[0:start]})

for i in range(start,end,step):
    print(i,i+step)
    X_train_features_temp = sess.run(model.logits, feed_dict={model.Image:X_train[i:i+step]})
    X_train_features = tf.concat([X_train_features,X_train_features_temp],0)
X_train_features_temp = sess.run(model.logits, feed_dict={model.Image:X_train[end:]})
X_train_features = tf.concat([X_train_features,X_train_features_temp],0)
X_train_features.shape
del X_train

sess.run(X_train_features)
X_train_features_np=X_train_features.eval(session=sess)
print(X_train_features_np.shape)
np.save('X_train_features.npy',X_train_features_np)
del X_train_features
del X_train_features_np

In [0]:
#@title Feature Extraction Validation Data
import numpy as np
import tflearn

start = 100
end = 600
step = 100
#(628, 299, 299, 3)
X_Val = np.load('X_Val.npy')
X_Val_features = sess.run(model.logits, feed_dict={model.Image:X_Val[0:start]})

for i in range(start,end,step):
    print(i,i+step)
    X_Val_features_temp = sess.run(model.logits, feed_dict={model.Image:X_Val[i:i+step]})
    X_Val_features = tf.concat([X_Val_features,X_Val_features_temp],0)
X_Val_features_temp = sess.run(model.logits, feed_dict={model.Image:X_Val[end:]})
X_Val_features = tf.concat([X_Val_features,X_Val_features_temp],0)
X_Val_features.shape
del X_Val

sess.run(X_Val_features)
X_Val_features_np=X_Val_features.eval(session=sess)
print(X_Val_features_np.shape)
np.save('X_Val_features.npy',X_Val_features_np)
del X_Val_features
del X_Val_features_np

In [0]:
X_aug = np.load('X_aug.npy')
print(X_aug.shape)

(1756, 299, 299, 3)


In [0]:
#@title Feature Extraction Augmentation Data
import numpy as np
import tflearn

start = 100
end = 1700
step = 100
#(1980, 299, 299, 3)
#X_aug = np.load('X_aug.npy')
#can't do it at once Memory Crash
X_aug_features = sess.run(model.logits, feed_dict={model.Image:X_aug[0:start]})

for i in range(start,end,step):
    print(i,i+step)
    X_aug_features_temp = sess.run(model.logits, feed_dict={model.Image:X_aug[i:i+step]})
    X_aug_features = tf.concat([X_aug_features,X_aug_features_temp],0)
X_aug_features_temp = sess.run(model.logits, feed_dict={model.Image:X_aug[end:]})
X_aug_features = tf.concat([X_aug_features,X_aug_features_temp],0)
X_aug_features.shape
del X_aug

sess.run(X_aug_features)
X_aug_features_np=X_aug_features.eval(session=sess)
print(X_aug_features_np.shape)
np.save('X_aug_features.npy',X_aug_features_np)
del X_aug_features
del X_aug_features_np

In [0]:
#@title Feature Extraction Test Data
import numpy as np
import tflearn

start = 100
end = 700
step = 100
#(772, 299, 299, 3)
X_test = np.load('X_test.npy')
X_test_features = sess.run(model.logits, feed_dict={model.Image:X_test[0:start]})

for i in range(start,end,step):
    print(i,i+step)
    X_test_features_temp = sess.run(model.logits, feed_dict={model.Image:X_test[i:i+step]})
    X_test_features = tf.concat([X_test_features,X_test_features_temp],0)
X_test_features_temp = sess.run(model.logits, feed_dict={model.Image:X_test[end:]})
X_test_features = tf.concat([X_test_features,X_test_features_temp],0)
X_test_features.shape
del X_test

sess.run(X_test_features)
X_test_features_np=X_test_features.eval(session=sess)
print(X_test_features_np.shape)
np.save('X_test_features.npy',X_test_features_np)
del X_test_features
del X_test_features_np

**Upload Extraced features and labels to drive , Not to do feature extration every time**

Executed only when we do feature extration

In [0]:
#@title Upload features & labels to drive { display-mode: "code" }
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials



# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
fid = '1xAboz-oYGj-KI2DBTdrCA98oi1aEH5lW'
'''f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": fid}]})
f.SetContentFile('X_train_features.npy')
f.Upload()

f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": fid}]})
f.SetContentFile('X_Val_features.npy')
f.Upload()

f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": fid}]})
f.SetContentFile('y_train.npy')
f.Upload()

f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": fid}]})
f.SetContentFile('y_Val.npy')
f.Upload()

f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": fid}]})
f.SetContentFile('String_labels.npy')
f.Upload()

f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": fid}]})
f.SetContentFile('X_test_features.npy')
f.Upload()

f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": fid}]})
f.SetContentFile('img_names.npy')
f.Upload()
'''
f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": fid}]})
f.SetContentFile('augmentedImages5.zip')
f.Upload()

f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": fid}]})
f.SetContentFile('y_aug.npy')
f.Upload()
f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": fid}]})
f.SetContentFile('X_aug_features.npy')
f.Upload()

# TensorFlow one fully connected layer model with train
Test Accuracy 84% (first submission)

In [0]:
#@title Load Features and Labels { display-mode: "code" }
import numpy as np
import tensorflow as tf

X_train_features = np.load('X_train_features.npy')
X_Val_features = np.load('X_Val_features.npy')

y_train = np.load('y_train.npy')
y_Val = np.load('y_Val.npy')

y_train_one_hot = tf.one_hot(y_train,10)
y_Val_one_hot = tf.one_hot(y_Val,10)

sess = tf.Session()
y_train=y_train_one_hot.eval(session=sess)
y_val = y_Val_one_hot.eval(session=sess)

print(X_train_features.shape)
print(y_train.shape)
print(X_Val_features.shape)
print(y_val.shape)

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mt
import tensorflow as tf

#this function is useful in order to delete old graphs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

height = 8
width = 8
channels = 1536
n_inputs = height*width

#parameters of fully connected network and outputs
n_fc1 = 1024
n_fc2 = 100
n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, height,width, channels], name = "X")
    y = tf.placeholder(tf.int32, shape = [None,n_outputs], name = "y")
    flat = tf.layers.flatten (X)   

with tf.name_scope("fc"):
    fc1 = tf.layers.dense(flat, n_fc1, activation = tf.nn.relu, name = "fc1")
    #fc2 = tf.layers.dense(fc1, n_fc2, activation = tf.nn.relu, name = "fc2")

with tf.name_scope("output"):
    dropout = tf.layers.dropout(fc1,rate=0.2,name="dropout")
    logits = tf.layers.dense(dropout, n_outputs, name = "output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=Y_proba, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.000001)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct =tf.equal(tf.argmax(Y_proba,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    #saver = tf.train.Saver()

n_epochs = 500
batch_size = 100
num_examples = 2510
sess = tf.Session()
with sess.as_default():
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(num_examples // batch_size):
            X_batch = X_train_features[iteration*batch_size:iteration*batch_size+batch_size,0:]
            y_batch = y_train[iteration*batch_size:iteration*batch_size+batch_size,0:]

            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_train_features, y: y_train})
        acc_test = accuracy.eval(feed_dict={X: X_Val_features, y: y_val})
        print("Epoch:",epoch+1, "Train accuracy:", acc_train, "test accuracy:", acc_test)


# Test and submission file

In [0]:
lebels_dictionary={
'airport_inside':	1,
'bakery':	2,
'bedroom':	3,
'greenhouse':	4,
'gym':	5,
'kitchen':	6,
'operating_room':	7,
'poolinside':	8,
'restaurant':	9,
'toystore':	10}

#Extracted Features from test data
X_test_features = np.load('X_test_features.npy')
#classes names ordered in correspondence to the one hot vector used in training phase 
String_labels = np.load('String_labels.npy')

labels_int=np.asarray([lebels_dictionary[i] for i in String_labels])
#test images names to be eritten in the csv file
img_names = np.load('img_names.npy')


X_test_features = np.load('X_test_features.npy')
String_labels = np.load('String_labels.npy')
img_names = np.load('img_names.npy')

feed_dict = {X:X_test_features }
classification = sess.run(Y_proba, feed_dict)# output of the softmax layer for the complete test Data
predictions_indices =  np.argmax(classification, axis=1)#get index of max probability for all test samples

#labels_int the index in it represent the label as the neural network does
#           the value  represents the labels as needed in the submission file (same as the announcement)
labels_int=np.asarray([lebels_dictionary[i] for i in String_labels])
predictions=labels_int[predictions_indices]#conversion between representations 

DAT =  np.column_stack((np.transpose(img_names),np.transpose(predictions)))
np.savetxt('submit.csv',(DAT),delimiter=",",header="id,label",  fmt="%s")

